In [1]:
import cv2
import pandas as pd
import datetime
import numpy as np
from PIL import Image
import os

In [33]:
## Calling the Classifiers

faceCascade = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

clf = cv2.face.LBPHFaceRecognizer_create()
clf.read("classifier.yml")

In [61]:
## Function to return the name of student based on the enrollment no
def user_name(user_id):
    df = pd.read_csv('student_data.csv')
    is_class, batch_list = return_batch_list()
    if is_class:
        for i in df.index:
            if df['Id'][i] == user_id:
                name = df['Name'][i]
#                 return name
                if does_batch_belong(batch_list,df['Batch'][i]):
                    name = df['Name'][i]
                    write_present(user_id,name)
                    return name
                else:
                    return 'Wrong Batch'
    else:
        return 'no_class'

In [49]:
## Marking the boundary of face in an image

def recognizer_boundary(img, classifier, scaleFactor, minNeighbours, color, clf):
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_img, scaleFactor, minNeighbours)
    coords = []
    for (x, y, w, h)in features:
        cv2.rectangle(img, (x,y),(x+w,y+h), color, 2)
        user_id, _ = clf.predict(gray_img[y:y+h, x:x+w])
        name = user_name(user_id)
#         print(name)
        cv2.putText(img, name, (x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
        coords = [x,y,w,h]
    return coords


In [7]:
## Function to call the recognizer function
def recognize(img, clf, faceCascade):
    color = {"blue":(255,0,0), "red":(0,0,255), "green":(0,255,0), "mix":(128,0,129)}
    coords = recognizer_boundary(img, faceCascade, 1.1, 10, color["green"], clf)
    return img

In [72]:
## Function to recognize the face of the student whose image has been captured

VideoCapture = cv2.VideoCapture(0)

while True:
    _,img = VideoCapture.read()
    img2 = recognize(img, clf, faceCascade)
    cv2.imshow("camera",img2)
    if cv2.waitKey(1) & 0xFF == ord('e'):
        break

VideoCapture.release()
cv2.destroyAllWindows()

In [3]:
def is_class_time(weekday,st,et):
    
    x = datetime.datetime.now()
    c_weekday = x.strftime("%A")
    
   
    time = x.strftime("%T")
    c_hour = int(time.split(":")[0])
    c_min = int(time.split(":")[1])
    
    
    st_hour = int(st.split(":")[0])
    st_min = int(st.split(":")[1])
    
    
    et_hour = int(et.split(":")[0])
    et_min = int(et.split(":")[1])
    
    
    if(weekday == c_weekday):
        
        if (c_hour >= st_hour) or ((c_hour == st_hour) & (c_min >= st_min)):
            
            if (c_hour < et_hour) or ((c_hour >= et_hour) & (c_min < et_min)):
                
                return True
            
            else:
                
                return False
            
        else:
            
            return False
        
    else:
        
        return False

In [57]:
def return_batch_list():
    
    df = pd.read_csv('trial.csv')
    
    check = 0
    pos = 0
    
    for i in df.index:
        if(is_class_time(df['Day'][i],df['Start Time'][i],df['End Time'][i])):
            check = 1
            pos = i
            break
    
    if check ==1:    
        return True, df['Batch'][pos]
    else:
        return False,''
        

In [12]:
def does_batch_belong(current_batches, stu_batch):
    batch = "'" + stu_batch +"'"
    if current_batches.find(batch) != -1:
        return True
    else:
        return False
    

In [70]:
def write_present(enroll, name):
    present_list = []
    time = datetime.datetime.now()
    entrytime = time.strftime('%T')
    df = pd.read_csv('present_students.csv')
    df = df.iloc[:,1:]
    for i in df.index:
        present_list.append(df['Id'][i])
    check = 0
    for i in present_list:
        if i == enroll:
            check = 1
            break
    if check == 0:
        df = df.append(dict(zip(df.columns,[enroll, name, entrytime])), ignore_index=True)
        df.to_csv('present_students.csv')